In [21]:
from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct
from numpy import array, linspace
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import plot
from scipy.signal import argrelextrema
from scipy.ndimage import gaussian_filter1d
import numpy as np
from miditoolkit.pianoroll import parser as pr_parser
from miditoolkit.pianoroll import utils
import matplotlib.pyplot as plt
import math

In [103]:
# helper functions
def read_midi(path,tpb=0):
    global DEFAULT_RESOLUTION
    mido_obj = mid_parser.MidiFile(path)
    tick_per_beat = mido_obj.ticks_per_beat
    if tpb==0:
        DEFAULT_RESOLUTION = tick_per_beat
    else:
        DEFAULT_RESOLUTION = tpb

    notes = [] 
    for instrument in mido_obj.instruments:
        for note in instrument.notes:
            #rescale tpb to 480
            note.start = int(note.start/tick_per_beat*DEFAULT_RESOLUTION)
            note.end=int(note.end/tick_per_beat*DEFAULT_RESOLUTION)
            notes.append(note)

    # sort by start time
    notes.sort(key=lambda note:note.start)
    return notes,DEFAULT_RESOLUTION

def write_midi(notes,centers_per_beat,path='out.mid',tick_per_beat=480,note_rh=None):
    out = mid_parser.MidiFile()
    out.ticks_per_beat = tick_per_beat
    out.instruments = [ct.Instrument(program=0,is_drum=False,name='right hand'),ct.Instrument(program=0,is_drum=False,name='left hand')]
    if note_rh is not None:
        for note in notes:
            assert(note.velocity)
            out.instruments[1].notes.append(ct.Note(start=note.start,end=note.end,pitch=note.pitch,velocity=note.velocity))
        for note in note_rh:
            assert(note.velocity)
            out.instruments[0].notes.append(ct.Note(start=note.start,end=note.end,pitch=note.pitch,velocity=note.velocity))
    else:
        for note in notes:
            assert(note.velocity)
            current_beat=note.start//tick_per_beat
            # centers = centers_per_beat[current_beat]
            closest_center,abs_distance_to_centers = cluster_assignment(note, centers_per_beat, tick_per_beat,single_note=True)
            if(closest_center==0):
                out.instruments[1].notes.append(ct.Note(start=note.start,end=note.end,pitch=note.pitch,velocity=note.velocity))
            else:
                out.instruments[0].notes.append(ct.Note(start=note.start,end=note.end,pitch=note.pitch,velocity=note.velocity))
    out.dump(path)

# def find_note_idx_start_before(notes, tick_per_beat, beat):
#     # binary search
#     notes = sorted(notes,key=lambda x:x.start)
#     target = beat*tick_per_beat-1
#     left=0
#     right=len(notes)-1
#     middle=(left+right+1)//2
#     while left<right:
#         if notes[middle].start == target:
#             if notes[middle+1].start==target:
#                 left=middle+1
#             else:
#                 break
#         elif notes[middle].start<target:
#             left=middle
#         else:
#             right=middle-1
#         middle=(left+right+1)//2
#     return middle

#note that are holding in this beat
def get_notes_of_beat(notes,beat,tick_per_beat):
    start_tick=beat*tick_per_beat+1
    end_tick=(beat+1)*tick_per_beat-1
    return list(filter(lambda n: n.start<=start_tick<=n.end or n.start<=end_tick<=n.end, notes))

#mode: normal, centroid, minmax
def get_centers_per_beat(notes, tick_per_beat,kernel_size=5,mode='normal'):
    # find centers
    centers_per_beat=[]
    range_per_beat=[]
    
    length = math.ceil(max(notes,key=lambda x:x.end).end/tick_per_beat)
    print(length,"debug")
    for beat in range(length):
        notes_within_window = get_notes_of_beat(notes,beat,tick_per_beat)
        pitches_within_window = np.array([note.pitch for note in notes_within_window])
        if len(pitches_within_window)==0:
            range_per_beat.append(None)
            continue
        upper_limit,lower_limit = max(pitches_within_window),min(pitches_within_window)
        range_per_beat.append([lower_limit,upper_limit])
    
    pitch_range = linspace(0,127)
    for beat in range(length):
        notes_within_window = get_notes_of_beat(notes,beat,tick_per_beat)
        pitches_within_window = np.array([note.pitch for note in notes_within_window])
        if len(pitches_within_window)==0:
            centers_per_beat.append(None)
            continue
            
        # use KDE for clustering(intervaling)(determine the center line for left/right hand respectively)
        pitches_within_window = pitches_within_window.reshape(-1,1)
        kernel_size=6
        #interatve clustering
        while True:
            kde = KernelDensity(kernel='gaussian', bandwidth=kernel_size).fit(pitches_within_window)
            kde_val = kde.score_samples(pitch_range.reshape(-1,1))
            # visualization: distribution of notes inside the window
            # plot(pitch_range, kde_val)
            minima, maxima = argrelextrema(kde_val, np.less)[0], argrelextrema(kde_val, np.greater)[0]
            
            
            #termination check
            if len(minima)>=1 and len(maxima)>=2:
                # cluster selection
                cutoff=min(pitch_range[minima])
                cluster_1_candidate = pitch_range[ maxima[pitch_range[maxima]<cutoff] ]
                cluster_2_candidate = pitch_range[ maxima[pitch_range[maxima]>cutoff] ]
                assert(len(cluster_1_candidate)>=1)
                assert(len(cluster_2_candidate)>=1)
                clusters= sorted([round(max(cluster_1_candidate)), round(max(cluster_2_candidate))])
                
                centers_per_beat.append(clusters)
                break
                
            elif kernel_size<0.000000000001:
                #probably only one hand is needed
                pitches_within_window.sort()
                if len(notes_within_window)>=3:
                    centers_per_beat.append([round(np.average(pitches_within_window[:len(pitches_within_window)//2])),
                                         round(np.average(pitches_within_window[len(pitches_within_window)//2:]))])
                else:
                    if round(np.average(pitches_within_window)) >=60:
                        centers_per_beat.append([round(np.average(pitches_within_window))-1,
                                         round(np.average(pitches_within_window))])
                    else:
                        centers_per_beat.append([round(np.average(pitches_within_window)),
                                                 round(np.average(pitches_within_window))+1])
                        
                break
            
            else:
                kernel_size*=0.7
                continue
            
        # diff modes
        if centers_per_beat[-1][0]>centers_per_beat[-1][1]:
            temp = centers_per_beat[-1][0]
            centers_per_beat[-1][0]=centers_per_beat[-1][1]
            centers_per_beat[-1][1]=temp
        centers = np.array(centers_per_beat[-1])
        left_candidate_notes=[]
        right_candidate_notes=[]
        
        for note in notes_within_window:
            abs_distance_to_centers=abs(note.pitch - centers)
            closest_center=np.argmin(abs_distance_to_centers) #0=left, 1 = right
            if closest_center==0:
                left_candidate_notes.append(note)
            else:
                right_candidate_notes.append(note)
                
        #DEBUG
        left_notes_pitch=np.array([n.pitch for n in left_candidate_notes])
        right_notes_pitch=np.array([n.pitch for n in right_candidate_notes])
#        print(left_candidate_notes,right_candidate_notes)

        if mode == "centroid":                    
            #left centroid
            if len(left_notes_pitch)>0:
                left_centroid=np.argmin(abs(left_notes_pitch-centers[0]))
                centers_per_beat[-1][0]=left_notes_pitch[left_centroid]            
            #right centroid
            if len(right_notes_pitch)>0:
                right_centroid=np.argmin(abs(right_notes_pitch-centers[1]))
                centers_per_beat[-1][1]=right_notes_pitch[right_centroid]
            
           # print(centers_per_beat[-1],left_notes_pitch,right_notes_pitch,notes_within_window)
                
        elif mode == "minmax":
            if len(left_notes_pitch)>0:
                left_centroid=np.argmin(left_notes_pitch)
                centers_per_beat[-1][0]=left_notes_pitch[left_centroid]
            if len(right_notes_pitch)>0:
                right_centroid=np.argmax(right_notes_pitch)
                centers_per_beat[-1][1]=right_notes_pitch[right_centroid]
            
                
    #swap center
    for i,center in enumerate(centers_per_beat):
        if center is not None:
            if center[0]>center[1]:
                temp=centers_per_beat[i][0]
                centers_per_beat[i][0]=centers_per_beat[i][1]
                centers_per_beat[i][1]=temp
                

    
    
                    
    #fill None
    for i,r in enumerate(range_per_beat):
        if r is None:
            left=None
            right=None
            for leftblk in reversed(range_per_beat[:i]):
                if leftblk is not None:
                    left=leftblk
                    break
            for rightblk in range_per_beat[i:]:
                if rightblk is not None:
                    right=rightblk
                    break
            if left is None:
                left=right
            if right is None:
                right=left
            assert(left is not None and right is not None) #the range_per_beat cannot be None
            range_per_beat[i]=np.array(left)+np.array(right)//2
    for i,r in enumerate(centers_per_beat):
        if r is None:
            left=None
            right=None
            for leftblk in reversed(centers_per_beat[:i]):
                if leftblk is not None:
                    left=leftblk
                    break
            for rightblk in centers_per_beat[i:]:
                if rightblk is not None:
                    right=rightblk
                    break
            if left is None:
                left=right
            if right is None:
                right=left
            assert(left is not None and right is not None) #the centers_per_beat cannot be None
            centers_per_beat[i]=np.array(left)+np.array(right)//2
                    
    
    #smooth the line
    centers_per_beat = np.array(centers_per_beat)
    centers_per_beat[:,0]=gaussian_filter1d(centers_per_beat[:,0], kernel_size)
    centers_per_beat[:,1]=gaussian_filter1d(centers_per_beat[:,1], kernel_size)
    
    
    assert(center_ordering(centers_per_beat))
    return np.array(centers_per_beat),np.array(range_per_beat)

def center_ordering(centers_per_beat):
        # make sure the cluster with lower octave stay at index 0 (left hand)
        for i,center in enumerate(centers_per_beat):
            if center is None:
                continue
            if center[0]>center[1]:
                print(i,center)
                return False
        else:
            return True
        
def cluster_assignment(notes, centers_per_beat, tick_per_beat, single_note=False):
    
    def assignment(note,centers_per_beat):
        current_beat=note.start//tick_per_beat
        centers = centers_per_beat[current_beat]
        abs_distance_to_centers=abs(note.pitch - centers)
        closest_center=np.argmin(abs_distance_to_centers)
        return closest_center,abs_distance_to_centers
    
    
    assert(center_ordering(centers_per_beat))
    if single_note == True:
        #return idx of cluster (0:left, 1:right)
        return assignment(notes,centers_per_beat)
    
    else:
        classified=[[],[]]
        abs_distance_to_classified_centers=[[],[]]
        for note in notes:
            closest_center,abs_distance_to_centers = assignment(note,centers_per_beat)
            classified[closest_center].append(note)
            abs_distance_to_classified_centers[closest_center].append(abs_distance_to_centers)            
        return classified,abs_distance_to_classified_centers

In [104]:
# filter long duration notes
def trim_long_notes(notes, tick_per_beat):
    def trim(note):
        if note.end-note.start>threshold*tick_per_beat:
            note.end = note.start+threshold*tick_per_beat
        return note
    
    threshold=4
    return list(map(trim, notes))

In [105]:
def octave_transpose(transpose_notes, tick_per_beat,threshold=10,center_threshold=5,minmaxrange=None,mode='normal',shiftbase=False):
    # octave transpose
    # for simplicity, use semitone instead of octave
    max_semitone_distance=threshold

    centers_per_beat,range_per_beat = get_centers_per_beat(transpose_notes, tick_per_beat,center_threshold,mode=mode)
    #use user provided range_per_beat
    if minmaxrange is not None:
        assert(len(minmaxrange)>=len(range_per_beat))
        range_per_beat=minmaxrange
        
    transpose_track=[]
    skip_track=[]
    count=0
    skip_count=0


    cached_note_beat=0
    cached_left_notes=[]
    cached_right_notes=[]
    all_left_notes, all_right_notes = [],[]
    shift_up_count=0
    for note_idx,note in enumerate(transpose_notes):
        # cluster assignment
        current_beat=note.start//tick_per_beat
        centers = centers_per_beat[current_beat]
        closest_center,abs_distance_to_centers = cluster_assignment(note, centers_per_beat, tick_per_beat,single_note=True)
        
        #---------------shift up(left hand)------------------------------
        #SHIFTBASE WITHOUT CHANGING THE NOTE ASSIGNMENT
        #print('---------------------------------------------')
        # print(note)
        # print(current_beat)
        # print(cached_left_notes)
        # print(cached_right_notes)
        if len(cached_left_notes)>0 and len(cached_right_notes)>0:
            #print('start shift up?')
            #assert all notes in the same beat
            assert( all([n[1].start//tick_per_beat == cached_left_notes[0][1].start//tick_per_beat for n in cached_left_notes]))
            assert( all([n[1].start//tick_per_beat == cached_right_notes[0][1].start//tick_per_beat for n in cached_right_notes]))
            assert( cached_left_notes[0][1].start//tick_per_beat == cached_right_notes[0][1].start//tick_per_beat)
                       
            
            if current_beat>cached_left_notes[0][1].start//tick_per_beat:
                #print('condition passed')
                left_highest_pitch=max(cached_left_notes, key=lambda n:n[1].pitch)[1].pitch
                right_lowest_pitch=min(cached_right_notes, key=lambda n:n[1].pitch)[1].pitch
                #print(left_highest_pitch,right_lowest_pitch,right_lowest_pitch-left_highest_pitch)
                if shiftbase and right_lowest_pitch-left_highest_pitch>18:
                    #print('shift up')
                    #print('shift up ',cached_left_notes[0][1].start//tick_per_beat,cached_left_notes,cached_right_notes)
                    for cached_note in cached_left_notes:
                        shift_up_count+=1
                        transpose_notes[cached_note[0]].pitch+=12
                        # all_left_notes[cached_note[0]].pitch += 12

                        
                        
        if len(cached_left_notes)>0 and current_beat>cached_left_notes[0][1].start//tick_per_beat:
            cached_left_notes=[]
        if len(cached_right_notes)>0 and current_beat>cached_right_notes[0][1].start//tick_per_beat:
            cached_right_notes=[]
        
        if closest_center==0:
            #print('added to left')
            all_left_notes.append(note)
            all_right_notes.append(None)
            cached_left_notes.append([note_idx,note])
        else:
            #print(note, abs_distance_to_centers, closest_center, max_semitone_distance)
            #print('added to right')
            all_left_notes.append(None)
            all_right_notes.append(note)
            cached_right_notes.append([note_idx,note])
        if shiftbase:
            continue
        #------------------end shift up----------------------------------
        
        if abs_distance_to_centers[closest_center]>max_semitone_distance:
            # transpose
            start_beat = (note.start)//tick_per_beat
            end_beat = (note.end-1)//tick_per_beat
            assert(start_beat<=end_beat)
            
            if note.pitch>centers[closest_center]:                
                # transpose downward                 
                if closest_center==0: 
                    #left hand
                    transpose_notes[note_idx].pitch-=12*( ((abs_distance_to_centers[closest_center]-max_semitone_distance)//12) +1)
                    #print(note,range_per_beat[start_beat:end_beat+1,0],start_beat,end_beat,range_per_beat)
                    if transpose_notes[note_idx].pitch<max(range_per_beat[start_beat:end_beat+1,0]):#the tranposed note can never go beyond the skyline
                        transpose_notes[note_idx].pitch+=12                        
                    else:
                        count+=1
                else:
                    #right hand
                    skip_count+=1
            else:                
                # transpose upward                
                if closest_center==1:
                    #print('transposing',transpose_notes[note_idx], abs_distance_to_centers, closest_center, max_semitone_distance)
                    #right hand
                    transpose_notes[note_idx].pitch+=12*( ((abs_distance_to_centers[closest_center]-max_semitone_distance)//12) +1)
                    #print('transposed!!!',transpose_notes[note_idx], abs_distance_to_centers, closest_center, max_semitone_distance, min(range_per_beat[start_beat:end_beat+1,1]))
                    if transpose_notes[note_idx].pitch>min(range_per_beat[start_beat:end_beat+1,1]):#the tranposed note can never go beyond the skyline
                        transpose_notes[note_idx].pitch-=12                      
                    else:
                        count+=1
                    #print('final@@@',transpose_notes[note_idx], abs_distance_to_centers, closest_center, max_semitone_distance)
                else:

                    #left hand
                    skip_count+=1

    print('Octave Transpose:','#notes',len(transpose_notes),'transposed',count,'notes',', skipped',skip_count,'shifted_up',shift_up_count)         
    #find the center again
    if shiftbase:
        centers_per_beat,range_per_beat = get_centers_per_beat(transpose_notes, tick_per_beat,center_threshold,mode=mode)
        all_left_notes = list(filter(lambda note: note is not None,all_left_notes))
        all_right_notes = list(filter(lambda note: note is not None,all_right_notes))
        # return transpose_notes,centers_per_beat,range_per_beat
        return all_left_notes,all_right_notes,centers_per_beat,range_per_beat
    
    # visualization : center line of left/right hand
    
    #plot(centers_per_beat)

    
    return transpose_notes,centers_per_beat,range_per_beat

In [106]:
x = None
x is None

True

In [107]:
def doubling_simplification(notes,centers_per_beat, tick_per_beat):
    notes = np.array(notes)
    count=0
    track_tick=0
    track_idx=0
    collections=[]
    delete=[]
    threshold=4 #max notes
    
    while track_idx<len(notes):
        if notes[track_idx].start==track_tick:
            # gathering notes start at the same tick
            collections.append(notes[track_idx])
            track_idx+=1
        else:
            # action
            classified,abs_distance_to_classified_centers = cluster_assignment(collections,centers_per_beat, tick_per_beat)
            keep=[[None for _ in range(12)] for __ in range(2)]
            # doubling simplification on different hand separately
            for group in range(2):
                note_count=0
                delete_candidates=[]
                for note in classified[group]:
                    if keep[group][note.pitch%12] is None:
                        keep[group][note.pitch%12] = note
                    elif keep[group][note.pitch%12].pitch>=note.pitch:
                        delete_candidates.append(note)
                    else:
                        delete_candidates.append(keep[group][note.pitch%12])
                        keep[group][note.pitch%12]=note
                    note_count+=1
                if note_count>threshold:
                    #remove doubling
                    delete_count = note_count-threshold
                    if group==0:
                        delete_candidates.sort(key=lambda x:x.pitch)
                    else:
                        delete_candidates.sort(key=lambda x:x.pitch,reverse=True)
                    delete_candidates = delete_candidates[0:delete_count]
                    for note in delete_candidates:
                        delete.append(note)
            # reset
            collections=[notes[track_idx]]
            track_tick=notes[track_idx].start
            track_idx+=1
            
    for note in delete:
        notes = np.delete(notes, np.where(notes == note)[0])
        count+=1
            
    print('Doubling Simplification:','removed doubling',count)
    return notes


In [108]:
def merge_discrete_note(notes,tick_per_beat):
    notes.sort(key=lambda note:note.start)   
    org_len=len(notes)
    #profile
    all_pitch={}
    for note in notes:
        pitch=note.pitch
        if pitch in all_pitch:
            all_pitch[pitch].append(note)
        else:
            all_pitch[pitch]=[note]
    #merge
    for k,v in all_pitch.items():
        v.sort(key=lambda note:note.start)   
        trace_idx=0
        while trace_idx<len(v)-1:
            if 0<=v[trace_idx+1].start-v[trace_idx].end<=merge_threshold*tick_per_beat:
                v[trace_idx].end=v[trace_idx+1].end
                v.pop(trace_idx+1)
            else:
                trace_idx+=1
    #output
    notes=[]
    for k,v in all_pitch.items():
        for note in v:
            notes.append(note)
    notes.sort(key=lambda note:note.start)      
    filtered_len=len(notes)
    print('Merge discrete:','merged ',org_len-filtered_len,' discrete notes')
    return notes

In [109]:
def drop_discrete_note(notes,tick_per_beat):
    org_len=len(notes)
    
    new_notes=[]
    for note in notes:
        if note.end-note.start > discrete_note_threshold*tick_per_beat:
            new_notes.append(note)
    notes=new_notes        
    print('Drop discrete:','dropped ',org_len-len(notes),' discrete notes')
    return notes

In [110]:
def visualize(notes,tick_per_beat,centers_per_beat=None,range_per_beat=None,length=-1):
    old_resol = tick_per_beat
    new_resol = 24
    factor_down = new_resol/ old_resol
    print(factor_down,new_resol,old_resol)
    
    notes=sorted(notes,key=lambda note:note.start)
    if old_resol>=new_resol:
        # dowmsample
        pianoroll = pr_parser.notes2pianoroll(
                            notes, 
                            binary_thres=1,
                            resample_factor=factor_down)
    else:
        # upsmaple
        pianoroll = pr_parser.pianoroll2notes(
                            pianoroll,
                            binary_thres=1,
                            resample_factor=1/factor_down)
        
    pianoroll = pianoroll.astype(bool)
    plt.figure(figsize = (20,20))
    legend=[]
    
    if length==-1:
        plt.imshow(pianoroll.T, origin='lower')                
        if centers_per_beat is not None:
            plt.plot(np.repeat(centers_per_beat[:, [1, 0]],new_resol , axis=0))
            legend=['Right hand','Left hand']
        if range_per_beat is not None:
            plt.plot(np.repeat(range_per_beat[:, [1, 0]],new_resol , axis=0))
            legend+=['upperlimit','lowerlimit']
    else:
        plt.imshow((pianoroll[:length]).T, origin='lower')
        if centers_per_beat is not None:
            plt.plot(np.repeat(centers_per_beat[:, 1],new_resol , axis=0)[:length],color="red")
            plt.plot(np.repeat(centers_per_beat[:, 0],new_resol , axis=0)[:length],color="white")
            legend=['Right hand','Left hand']
        if range_per_beat is not None:
            plt.plot(np.repeat(range_per_beat[:, [1, 0]],new_resol , axis=0)[:length])
            legend+=['upperlimit','lowerlimit']
    plt.legend(legend)

    plt.show()

In [111]:
# main

def main(path,outpath):
    merge_threshold=0.0001# #minimum separation between notes in same pitch (%of bpt)
    discrete_note_threshold=0.05 #minimum length of a note(%of bpt)

    notes,tick_per_beat = read_midi(path)
    org_notes=len(notes)
    print(f'before post-processing {org_notes} notes')
    notes = trim_long_notes(notes, tick_per_beat)
    #notes,centers_per_beat,range_per_beat = octave_transpose(notes, tick_per_beat,mode='centroid')
    notes,centers_per_beat,range_per_beat = octave_transpose(notes, tick_per_beat,1,10,mode='centroid')
    #notes = merge_discrete_note(notes,tick_per_beat)
    notes = drop_discrete_note(notes,tick_per_beat)
    notes = doubling_simplification(notes,centers_per_beat, tick_per_beat)
    cur_notes=len(notes)
    print(f'after post-processing, total notes: {cur_notes}, removed {org_notes-cur_notes} notes in total')
    visualize(notes,tick_per_beat,centers_per_beat=centers_per_beat,range_per_beat=None,length=5000)
    write_midi(notes,centers_per_beat, outpath, tick_per_beat)

    minimum_runs=3
    run=0
    while True:
        notes,tick_per_beat = read_midi(outpath)
        org_notes=len(notes)
        print(f'before post-processing {org_notes} notes')
        notes = trim_long_notes(notes, tick_per_beat)
        notes,centers_per_beat,range_per_beat = octave_transpose(notes, tick_per_beat,1,10,range_per_beat,mode='centroid')
        #notes = merge_discrete_note(notes,tick_per_beat)
        notes = drop_discrete_note(notes,tick_per_beat)
        notes = doubling_simplification(notes,centers_per_beat, tick_per_beat)
        cur_notes=len(notes)
        print(f'after post-processing, total notes: {cur_notes}, removed {org_notes-cur_notes} notes in total')
        visualize(notes,tick_per_beat,centers_per_beat=centers_per_beat,range_per_beat=None,length=5000)
        write_midi(notes,centers_per_beat, outpath, tick_per_beat)
        run+=1
        #termination
        if org_notes-cur_notes==0 and run>=minimum_runs:        
            break
            
    notes,tick_per_beat = read_midi(outpath)
    org_notes=len(notes)
    print(f'before post-processing {org_notes} notes')
    notes_lh,notes_rh, centers_per_beat,range_per_beat = octave_transpose(notes, tick_per_beat,0,1,range_per_beat,mode='minmax',shiftbase=True)
    # notes, centers_per_beat,range_per_beat = octave_transpose(notes, tick_per_beat,0,1,range_per_beat,mode='minmax',shiftbase=True)
    cur_notes=len(notes)
    print(f'after post-processing, total notes: {cur_notes}, removed {org_notes-cur_notes} notes in total')
    visualize(notes,tick_per_beat,centers_per_beat=centers_per_beat,range_per_beat=None,length=5000)
    write_midi(notes_lh,centers_per_beat, outpath, tick_per_beat,centers_per_beat,notes_rh)
# write_midi(notes,"post_processed.mid", tick_per_beat)

In [112]:
main("test_r2flarge.mid","post_processed.mid")

before post-processing 513 notes
129 debug
Octave Transpose: #notes 513 transposed 16 notes , skipped 123 shifted_up 0
Drop discrete: dropped  0  discrete notes


TypeError: assignment() missing 1 required positional argument: 'centers_per_beat'

In [249]:
centers_per_beat

array([[44, 73],
       [43, 73],
       [43, 73],
       [43, 73],
       [42, 73],
       [42, 74],
       [41, 74],
       [40, 74],
       [40, 74],
       [39, 74],
       [38, 74],
       [37, 74],
       [37, 74],
       [36, 74],
       [35, 73],
       [35, 72],
       [35, 72],
       [35, 71],
       [35, 70],
       [35, 69],
       [35, 68],
       [36, 68],
       [37, 67],
       [38, 66],
       [39, 66],
       [40, 65],
       [42, 65],
       [43, 65],
       [44, 65],
       [45, 65],
       [46, 65],
       [47, 65],
       [48, 65],
       [49, 66],
       [50, 66],
       [51, 66],
       [51, 66],
       [52, 67],
       [52, 67],
       [52, 67],
       [53, 67],
       [53, 68],
       [53, 68],
       [53, 68],
       [54, 69],
       [54, 70],
       [55, 70],
       [55, 71],
       [56, 72],
       [56, 73],
       [57, 74],
       [57, 76],
       [58, 77],
       [58, 78],
       [59, 79],
       [59, 79],
       [59, 80],
       [59, 81],
       [59, 81

In [250]:
range_per_beat

array([[48, 76],
       [48, 76],
       [48, 76],
       [45, 78],
       [45, 78],
       [45, 78],
       [40, 79],
       [40, 79],
       [40, 79],
       [36, 81],
       [36, 81],
       [36, 81],
       [30, 82],
       [30, 82],
       [30, 82],
       [31, 83],
       [31, 83],
       [31, 83],
       [29, 71],
       [29, 71],
       [29, 71],
       [28, 76],
       [28, 76],
       [28, 76],
       [40, 64],
       [36, 64],
       [64, 64],
       [40, 66],
       [33, 66],
       [66, 66],
       [40, 67],
       [40, 67],
       [67, 67],
       [40, 69],
       [36, 69],
       [69, 69],
       [40, 70],
       [30, 70],
       [70, 70],
       [40, 71],
       [31, 71],
       [71, 71],
       [47, 59],
       [29, 59],
       [59, 59],
       [40, 64],
       [28, 76],
       [64, 74],
       [64, 73],
       [33, 69],
       [64, 71],
       [66, 72],
       [33, 86],
       [66, 84],
       [67, 83],
       [40, 93],
       [57, 91],
       [50, 90],
       [38, 86

In [ ]:
list(filter(lambda x:x.start>=91136,notes))

In [ ]:
len(centers_per_beat)

In [ ]:
cpb,rpb = get_centers_per_beat(notes, tick_per_beat,5,mode='centroid')

In [ ]:
len(range_per_beat)

# DEBUG

In [ ]:
    for beat in range(10):
        notes_within_window = get_notes_of_beat(notes,beat,tick_per_beat)
        pitches_within_window = np.array([note.pitch for note in notes_within_window])
        upper_limit,lower_limit = max(pitches_within_window),min(pitches_within_window)
        range_per_beat.append([lower_limit,upper_limit])

In [ ]:
octave_transpose(notes, tick_per_beat)

In [ ]:
centers_per_beat=[]
range_per_beat=[]

ahead=1
after=1

start=ahead
end=(notes[-1].start//tick_per_beat - after) +1

trace_start_idx=0

for beat in range(end):
    notes_within_window = notes[trace_start_idx:find_note_idx_before(notes, tick_per_beat, beat+1)+1]
    print(notes_within_window)
    pitches_within_window = np.array([note.pitch for note in notes_within_window])
    upper_limit,lower_limit = max(pitches_within_window),min(pitches_within_window)
    range_per_beat.append([lower_limit,upper_limit])
    trace_start_idx+=find_note_idx_before(notes, tick_per_beat, beat+1)+1
range_per_beat

In [ ]:
notes

In [ ]:
notes